## Quantum Teleportation
This algorithm performs the quantum teleportation of a quantum state from one person, Alice, to another, Bob. Here is the general scheme:

1. We start with three qubits:

    * One initialized in the quantum state we want to teleport.

    * Two others prepared in an entangled state shared between Alice and Bob.

2. A CNOT gate is applied between the qubit to be teleported (as control) and Alice's entangled qubit (as target). Then, a Hadamard gate is applied to the qubit we want to teleport.

3. The teleportation is completed by performing measurements on Alice’s two qubits (the one in the initial state and her half of the entangled pair). The result of these measurements is sent to Bob, who applies the appropriate quantum gates to his qubit in order to recover the original state.


![Alt text](quantum_teleportation.png)


In [129]:
import numpy as np
import matplotlib.pyplot as plt

In [130]:
import cirq

In [147]:
# les qubits
state_to_teleport = cirq.NamedQubit('state_to_teleport')
alice = cirq.NamedQubit('Alice')
bob = cirq.NamedQubit('Bob')




# I create the state that i want to teleport
message = cirq.Circuit(
    cirq.X(state_to_teleport) ** 1,
    cirq.Y(state_to_teleport) ** 0.5, 
)


teleport_circuit= cirq.Circuit(
    # I create the entangled state between Alice and Bob
    cirq.H(alice),
    cirq.CNOT(alice, bob),

    # message circuit
    message,
    
    # teleportation protocol
    cirq.CNOT(state_to_teleport, alice),
    cirq.H(state_to_teleport),
    cirq.measure(state_to_teleport ,  key='M_1'),
    cirq.measure(alice , key='M_2'),

    # I apply the correction to Bob's qubit based on the measurement results
    cirq.X(bob).with_classical_controls('M_1'),
    cirq.Z(bob).with_classical_controls('M_2'),
)
print(teleport_circuit)


simulator = cirq.Simulator()

state_vector_teleported = cirq.bloch_vector_from_state_vector(
    simulator.simulate(message).final_state_vector, 0
)
teleport_bloch_vector = cirq.bloch_vector_from_state_vector(
    simulator.simulate(teleport_circuit).final_state_vector, 1
)

print(f"Message Qubit State: {state_vector_teleported}")
print(f"Teleported Bob's Qubit state: {teleport_bloch_vector}")



                                              ┌──┐
Alice: ───────────────H───@───────────────X─────M────────────────
                          │               │     ║
Bob: ─────────────────────X───────────────┼─────╫────────X───Z───
                                          │     ║        ║   ║
state_to_teleport: ───────────X───Y^0.5───@────H╫────M───╫───╫───
                                                ║    ║   ║   ║
M_1: ═══════════════════════════════════════════╬════@═══^═══╬═══
                                                ║            ║
M_2: ═══════════════════════════════════════════@════════════^═══
                                              └──┘
Message Qubit State: [-1.  0.  0.]
Teleported Bob's Qubit state: [-1.  0.  0.]
